In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy
CC = ComplexField(32)
pi = CC.pi()
I = CC.gen()

def generate_random_hnp_instance(leak_size, number_of_signatures, prime, key):
    HNP_instance = []
    bound = (prime-1) / 2**(leak_size+1)

    for j in range(number_of_signatures):
        c = randrange(1, prime)
        k = randrange(-round(bound), floor(bound))
        h = (k - c*key) % prime
        HNP_instance.append([c, h])
    return HNP_instance

def sort_and_difference(hnp_instance, prime, number_of_iterations, bound_exponent):
    new_instance = hnp_instance
    for i in range(number_of_iterations):
        new_instance.sort(key=lambda x:x[0])
        helper_list = []
        for j in range(len(new_instance)-1):
            if i < len(new_instance)-2 | (new_instance[j+1][0]-new_instance[j][0]) < 2**bound_exponent:
                helper_list.append(((new_instance[j+1][0]-new_instance[j][0]), (new_instance[j+1][1]-new_instance[j][1])% prime))
        new_instance = helper_list.copy()
    return new_instance

def bleichenbacher(hnp_instance, prime, bound_exponent):
    z = [0] * (2**bound_exponent + 1)
    for i in range(len(hnp_instance)-1):
        z[hnp_instance[i][0]] += (1/len(hnp_instance)) * exp(2*pi*I* hnp_instance[i][1]/prime)
    frequencies = numpy.fft.ifft(z)
    magnitudes = numpy.abs(frequencies)
    
    return numpy.argmax(magnitudes)*prime/(2**(bound_exponent))

def transform_hnp_instance_to_lattice_reduction_form(hnp_instance, prime):
    for i in range(len(hnp_instance)):
        hnp_instance[i][0] = (-1 * hnp_instance[i][0]) % prime

def nearest_plane_recursive(basis, GM_basis, x, dimension):
    dimension -= 1

    if dimension == 0:
        delta = round(x *  GM_basis.column(dimension))
        return delta * basis.column(dimension)

    gamma_d = x *  GM_basis.column(dimension)
    
    delta = round(gamma_d)
    
    v = delta * basis.column(dimension)
    x_prime = x - v
    
    y = nearest_plane_recursive(basis, GM_basis, x_prime, dimension)
    
    return y + v




In [117]:
import math

LEAK_SIZE = 1
NUMBER_OF_SIGNATURES = 5000
NUMBER_OF_RECOVERED_MSB = 15 #maximalni velikost fft

prime = 600979  
key = 500000

print(math.log2(key))

hnp_instance = generate_random_hnp_instance(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key)

hnp_instance = sort_and_difference(hnp_instance, prime, 2, NUMBER_OF_RECOVERED_MSB)

msb_of_key = bleichenbacher(hnp_instance, prime, NUMBER_OF_RECOVERED_MSB)

print(msb_of_key)
print("the original key is: ", bin(key))
print("we recovered:", bin(int(round(key - msb_of_key))))

18.931568569324174
490587.99044799805
the original key is:  0b1111010000100100000
we recovered: 0b10010011000100


In [ ]:

LEAK_SIZE = 5
NUMBER_OF_SIGNATURES = 200

prime = 33233
key = 30000

hnp_instance = generate_random_hnp_instance(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key)

transform_hnp_instance_to_lattice_reduction_form(hnp_instance, prime)


basis = matrix(len(hnp_instance) + 1)
basis[len(hnp_instance), len(hnp_instance)] = 1
for i in range(len(hnp_instance)):
    basis[i,i] = prime * prime
    basis[len(hnp_instance), i] = hnp_instance[i][0]*prime


reduced_basis = basis.LLL(delta=3/4)
reduced_basis = reduced_basis * (1/prime)

target = zero_vector(RDF, len(hnp_instance)+1)
for i in range(len(hnp_instance)):
    target[i] = hnp_instance[i][1]



reduced_basis = reduced_basis.transpose()
reduced_basis = reduced_basis.change_ring(RDF)
gramm_schmidt_basis, coefficients = reduced_basis.QR()

#nearest plane
b = vector(target)

for i in range(len(b)-1, -1, -1):
    b = b - reduced_basis.column(i) * round(b * gramm_schmidt_basis.column(i))
    
print(target - b)


(5.5453888439785347e+73, -1.293413066652456e+72, -1.317011327262346e+74, 1.4250332337420564e+74, -6.145761235391087e+73, -4.64084522776035e+73, -3.5786801054569994e+73, -5.539817357138404e+73, -5.5365732553412916e+73, -7.742006044875815e+73, 1.3548898838762684e+74, -8.000265892987037e+73, -2.4604485358486344e+73, -6.311281613359404e+73, 4.655232303237724e+73, -8.967858111639292e+73, 1.4161330920609597e+74, 3.608370973165842e+73, 1.2123817080401967e+73, 1.0832517839845846e+74, -1.399489382588719e+74, -1.389150533359884e+74, 7.745744098207754e+73, 1.2376857020576768e+74, -3.593983897688467e+73, 1.14746407894016e+74, -1.380525406705016e+74, -1.3359259079925658e+74, -9.07956824103005e+73, 1.261424158961639e+74, -5.422535740907515e+73, 5.357159753430431e+73, 2.2527548345178532e+73, 1.3201637573840495e+74, -7.943705494147197e+73, 6.970541788654817e+73, 7.698916004759253e+73, 1.1326186450857381e+73, -7.955765184581555e+73, 1.0186234071738884e+74, -5.3974996432847074e+73, -1.3942494245598555e+